In [ ]:
#-*- coding: utf-8 -*-
# 라이브러리 import
import sys

# install chromium, its driver, and selenium
if 'google.colab' in sys.modules:
    !apt-get update
    !apt install chromium-chromedriver
    !cp /usr/lib/chromium-browser/chromedriver /usr/bin
    !pip install selenium

# set options to be headless, ..
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

if 'ipykernel' in sys.modules:
    from tqdm import tqdm_notebook as tqdm
else:
    from tqdm import tqdm

import datetime
from dateutil.relativedelta import relativedelta
try:
    from urllib.parse import urlencode
except:
    !pip install urllib3
    from urllib.parse import urlencode

import time
import calendar

try:
    import csv
except:
    !pip install csv
    import csv

In [ ]:
# 브라우저 옵션
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument(
            "user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36"
        )
options.add_argument("lang=ko_KR")

In [ ]:
# 루프 제한 값
sleepMaxCount = 5
# select query
item_query = "ytd-video-renderer.style-scope.ytd-item-section-renderer"
title_query = "div#title-wrapper yt-formatted-string.style-scope.ytd-video-renderer"
like_query = "ytd-video-meta-block.style-scope.ytd-video-renderer.byline-separated > div#metadata > div#metadata-line > span"
time_query = "ytd-video-meta-block.style-scope.ytd-video-renderer.byline-separated > div#metadata > div#metadata-line > span"
channel_query = "ytd-channel-name#channel-name.long-byline,style-scope,ytd-video-renderer > div#container > div#text-container > yt-formatted-string.style-scope.ytd-channel-name > a"

In [ ]:
# 데이터 추출 함수
def startSearch() :
    search_txt = input("search : ")
    encode_type = "utf-8"

    sq={"search_query":search_txt}
    search_encode = urlencode(sq)

    if __name__ == '__main__' or 'google.colab' in sys.modules:
        chrome = webdriver.Chrome('chromedriver', options=options)
        try:
            chrome.get("https://www.youtube.com/results?" + search_encode + "&sp=CAISAggF")
            item_query = "ytd-video-renderer.style-scope.ytd-item-section-renderer"
            i=0
            dataRuw=0
            isEnd=0
            time.sleep(1)

            # 초기 파일 생성
            now_date = datetime.datetime.now()
            cvsFileName = "youtube_search_" + search_txt + "_" + now_date.strftime("%Y-%m-%d") + ".csv"
            with open(cvsFileName, "w", newline="", encoding=encode_type) as f:
                colnames = ['number', 'title', 'like', 'time', "channel"]
                w = csv.writer(f)
                w.writerow(colnames)

            # 데이터 추출을 위한 루프
            while isEnd < sleepMaxCount:
                isEnd = 0
                items = chrome.find_elements_by_css_selector(item_query)
                item_len=items.__len__()

                # 최초 데이터 로드 될때까지 루프, 최대 sleepMaxCount 까지만 루프
                while item_len==0 and isEnd < sleepMaxCount:
                    time.sleep(1)
                    items = chrome.find_elements_by_css_selector(item_query)
                    item_len = items.__len__()
                    isEnd = isEnd + 1

                # 최대 sleepMaxCount 루프까지 진행 된게 아이라면 isEnd 초기화
                if isEnd < sleepMaxCount:
                    isEnd = 0

                # 스크롤에 의한 데이터 로드 체크 및 추출 루프
                while item_len <= dataRuw and isEnd < sleepMaxCount:
                    # 브라우저 스크롤 맨 아래로 이동
                    chrome.execute_script("window.scrollTo(0, document.getElementById('content').scrollHeight);")
                    time.sleep(1)
                    items = chrome.find_elements_by_css_selector(item_query)
                    item_len = items.__len__()
                    with open(cvsFileName, "a", newline="", encoding=encode_type) as f:
                        w = csv.writer(f)
                        number = 0
                        for item in items[dataRuw:]:
                            number=number+1
                            title_txt = ""
                            like_txt = ""
                            time_txt = ""
                            channel_txt = ""
                            try:
                                title_txt = item.find_element_by_css_selector(title_query).text
                            except:
                                print("title_txt error")
                                title_txt = ""
                            
                            try:
                                like_txt = item.find_elements_by_css_selector(like_query)[0].text
                            except:
                                print("like_txt error")
                                like_txt = ""
                                
                            try:
                                time_txt = item.find_elements_by_css_selector(time_query)[1].text
                            except:
                                print("time_txt error")
                                time_txt = ""

                            try:
                                channel_txt = item.find_element_by_css_selector(channel_query).text
                            except:
                                print("channel_txt error")
                                channel_txt = ""

                            #if encode_type == "euc_kr":
                            #    title_txt = title_txt.encode('euc_kr','replace').decode("euc_kr")
                            data = [number, title_txt, like_txt, time_txt, channel_txt]
                            print(data)
                            w.writerow(data)
                    isEnd = isEnd + 1
                    time.sleep(1)

                dataRuw = item_len
                i = i+1
                print("스크롤 : ", i)
                print("데이터 건수 : ", dataRuw)
        except Exception as e:
            print(e)
        finally:
            if chrome:
                chrome.close()


In [ ]:
startSearch()